In [11]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Загрузка данных
data = pd.read_csv("../data/processed/train.csv")
data

,total_meters,price
0,40.1,35500000
1,79.0,75000000
2,21.6,4700000
3,51.7,15000000
4,43.0,30800000
...,...,...
593,31.9,17490000
594,37.0,14950000
595,38.0,11500000
596,32.5,12930000


In [15]:
# Стандартизация признака
scaler = StandardScaler()
data['total_meters'] = scaler.fit_transform(data[['total_meters']])

# Разделение на признаки и целевую переменную
X = data[['total_meters']]  # только один признак - площадь
y = data['price']

# Убираем крайние выбросы
q_low, q_high = y.quantile([0.01, 0.99])
mask = (y >= q_low) & (y <= q_high)
X, y = X[mask], y[mask]

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Лог-трансформация целевой
y_train_log = np.log1p(y_train)

# Создание и обучение модели
# Определение Pipeline и GridSearchCV
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Ridge())
])

param_grid = [
    {'model': [Ridge()], 'model__alpha': np.logspace(-3, 3, 13)},
    {'model': [RandomForestRegressor(random_state=42)],
     'model__n_estimators': [50, 100], 'model__max_depth': [None, 10, 20]},
    {'model': [GradientBoostingRegressor(random_state=42)],
     'model__n_estimators': [50, 100], 'model__learning_rate': [0.01, 0.1], 'model__max_depth': [3, 5]}
]

model = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Обучаем и выбираем лучшее α
model.fit(X_train, y_train_log)
best_model = model.best_estimator_
print(f"Лучшие гиперпараметры: {model.best_params_}")
print(f"CV R2: {model.best_score_:.3f}")

# Предсказание на тестовой выборке
y_pred_log = best_model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # обратное exp

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
# todo: use logging
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

# Коэффициенты модели
model_step = best_model.named_steps['model']
if hasattr(model_step, 'coef_'):
    coef = model_step.coef_
    print(f"Коэффициент при площади: {coef[0]:.2f}")
    print(f"Свободный член: {model_step.intercept_:.2f}")
else:
    print("Выбранная модель не поддерживает атрибут coef_")

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Лучшие гиперпараметры: {'model': RandomForestRegressor(random_state=42), 'model__max_depth': None, 'model__n_estimators': 100}
CV R2: 0.859
Среднеквадратичная ошибка (MSE): 21466511084498.04
Корень из среднеквадратичной ошибки (RMSE): 4633196.64
Коэффициент детерминации R²: 0.859277
Средняя ошибка предсказания: 2082725.61 рублей
Выбранная модель не поддерживает атрибут coef_


In [16]:
import joblib
# Сохранение модели
model_path = '../models/best_regression_v1.pkl'

joblib.dump(best_model, model_path)
print(f"Модель сохранена в файл {model_path}")

# Загрузка модели
loaded_model = joblib.load(model_path)
print("Модель загружена из файла")

Модель сохранена в файл ../models/best_regression_v1.pkl
Модель загружена из файла
